In [75]:
import pandas as pd
import os
from bs4 import BeautifulSoup

In [76]:
# Import the Rotten Tomatoes .tsv into a pandas datafram
df1 = pd.read_csv('bestofrt.tsv', sep = '\t')

In [77]:
df1.head()

,ranking,critic_score,title,number_of_critic_ratings
0,1,99,The Wizard of Oz (1939),110
1,2,100,Citizen Kane (1941),75
2,3,100,The Third Man (1949),77
3,4,99,Get Out (2017),282
4,5,97,Mad Max: Fury Road (2015),370


In [78]:
# working through methods below

with open('rt_html/et_the_extraterrestrial.html') as file:
    soup = BeautifulSoup(file, 'lxml')
    score = soup.find(class_ = 'audience-info hidden-xs superPageFontColor')
    print(score.find_all('div')[1].contents[-1].strip('\n').strip(' ').replace(',', ''))
    
    

32313030


In [79]:


# Create dictionary for movies
df_list = []
folder = 'rt_html'
for movie_html in os.listdir(folder):
    with open(os.path.join(folder, movie_html)) as file:
        soup = BeautifulSoup(file, 'lxml')
        # Get the title
        title = soup.find('title').contents[0].split(' -')[0].replace(u'\xa0', u' ')
        # Get the user score and number
        score = soup.find(class_ = 'audience-score meter')
        audience_score = score.find('span').string.replace('%', '')
        
        number = soup.find(class_ = 'audience-info hidden-xs superPageFontColor')
        num_audience_ratings = number.find_all('div')[1].contents[-1].strip('\n').strip(' ').replace(',', '')
        # Append to list of dictionaries
        df_list.append({'title': title,
                        'audience_score': int(audience_score),
                        'number_of_audience_ratings': int(num_audience_ratings)})
df2 = pd.DataFrame(df_list, columns = ['title', 'audience_score', 'number_of_audience_ratings'])

In [85]:
df1.sort_values(by='title', inplace=True)
df2.sort_values(by='title', inplace=True)



pandas.core.series.Series

In [97]:
for title in df1.title:
    if title not in list(df2.title):
        print(title)
        
for title in df2.title:
     if title not in list(df1.title):
         print(title)

Army of Shadows (L'Armée des ombres) (1969)
Harry Potter and the Deathly Hallows - Part 2 (2011)
Rashômon (1951)
Star Wars: Episode VII - The Force Awakens (2015)
Tokyo Story (Tôkyô monogatari) (1953)
Army of Shadows (L'ArmÃ©e des ombres) (1969)
Harry Potter and the Deathly Hallows
RashÃ´mon (1951)
Star Wars: Episode VII
Tokyo Story (TÃ´kyÃ´ monogatari) (1953)


In [ ]:
df2.replace(to_replace="Army of Shadows (L'ArmÃ©e des ombres) (1969)", value="Army of Shadows (L'Armée des ombres) (1969)", inplace=True)
df2.replace(to_replace="Harry Potter and the Deathly Hallows", value="Harry Potter and the Deathly Hallows - Part 2 (2011)", inplace=True)
df2.replace(to_replace="RashÃ´mon (1951)", value="Army of Shadows (L'Armée des ombres) (1969)", inplace=True)
df2.replace(to_replace="Army of Shadows (L'ArmÃ©e des ombres) (1969)", value="Army of Shadows (L'Armée des ombres) (1969)", inplace=True)
df2.replace(to_replace="Army of Shadows (L'ArmÃ©e des ombres) (1969)", value="Army of Shadows (L'Armée des ombres) (1969)", inplace=True)


In [80]:
df = pd.merge(df1, df2, on=['title'], how='outer')

In [81]:
df

,ranking,critic_score,title,number_of_critic_ratings,audience_score,number_of_audience_ratings
0,1.0,99.0,The Wizard of Oz (1939),110.0,89.0,874425.0
1,2.0,100.0,Citizen Kane (1941),75.0,90.0,157274.0
2,3.0,100.0,The Third Man (1949),77.0,93.0,53081.0
3,4.0,99.0,Get Out (2017),282.0,87.0,63837.0
4,5.0,97.0,Mad Max: Fury Road (2015),370.0,86.0,123937.0
5,6.0,100.0,The Cabinet of Dr. Caligari (Das Cabinet des D...,49.0,89.0,27163.0
6,7.0,100.0,All About Eve (1950),64.0,94.0,44564.0
7,8.0,98.0,Inside Out (2015),324.0,89.0,133558.0
8,9.0,99.0,The Godfather (1972),85.0,98.0,730433.0
9,10.0,99.0,Metropolis (1927),116.0,92.0,62018.0
